In [ ]:
import pathlib

import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
seq_id = 39031
seq_dir_name = f'MVI_{seq_id}'

response_map_vis_dir = pathlib.Path('./response_map_vis')
response_map_data_seq_dir = pathlib.Path(response_map_vis_dir) / seq_dir_name

dataset_dir = pathlib.Path('../../datasets/UA-DETRAC_GluonCV')
images_dir = dataset_dir / 'raw_data' / 'Insight-MVT_Annotation_Test' / seq_dir_name

In [ ]:
frame_num = 1
image_file = images_dir / f'img{frame_num:05d}.jpg'
print(image_file)
image = cv.imread(str(image_file), cv.IMREAD_COLOR)
frame_text = f'frame_{frame_num:04d}_'
response_map_file = response_map_data_seq_dir / (frame_text + 'response_map.npy')
boxes_file = response_map_data_seq_dir / (frame_text + 'boxes.npy')
response_maps = np.load(response_map_file)
boxes = np.load(boxes_file)
image.shape, response_maps.shape, boxes.shape

In [ ]:
def image_show(image):
    plt.imshow(image[:,:,::-1])

image_show(image)

In [ ]:
def resize_boxes(boxes_xyxy, old_size, new_size):
    scale_xy = np.asarray(new_size) / np.asarray(old_size)
    scale = np.concatenate((scale_xy, scale_xy))
    boxes_resized = boxes_xyxy * scale
    return boxes_resized

boxes = resize_boxes(boxes, (960, 512), (960, 540))
boxes = boxes.round().astype(int).tolist()
boxes

In [ ]:
def draw_box(image, box):
    x1, y1, x2, y2 = box
    cv.rectangle(image, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=1, lineType=cv.LINE_AA)

def draw_response_map(image, box, response_map, reduction=np.mean, upscale=16):
    response_map_reduced = reduction(response_map, axis=0, keepdims=True)
    response_map_reduced = np.transpose(response_map_reduced, (1, 2, 0))
    height, width, _ = response_map_reduced.shape
    new_height, new_width = height * upscale, width * upscale
    response_map_resized = cv.resize(response_map_reduced, (new_height, new_width), interpolation=cv.INTER_CUBIC)
    response_map_norm = ((response_map_resized / np.max(response_map_resized)) * 255).round().astype(np.uint8)
    response_cmap = cv.applyColorMap(response_map_norm, cv.COLORMAP_JET)
    return response_cmap

for box in boxes:
    draw_box(image, box)

# image_show(image)
rm_cmap = draw_response_map(image, box, response_maps[0])
image_show(rm_cmap)

In [ ]:
from response_map_vis import (
    create_response_heatmaps, stack_response_heatmaps, draw_heatmaps_comparison
)

out_size = 64
response_heatmaps = create_response_heatmaps(response_maps)
heatmaps_stacked = stack_response_heatmaps(
    response_heatmaps, 'horizontal', out_height=out_size, out_width=out_size
)
frame = draw_heatmaps_comparison(image, boxes, boxes, heatmaps_stacked, heatmaps_stacked)
image_show(frame)